In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install keras_preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.8 MB/s eta 0:00:00


In [2]:
import os
import zipfile
import shutil
from keras_preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Concatenate,GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model, load_model, Sequential
import random

In [ ]:
import zipfile

# Specify the path to the ZIP file
zip_file_path = '/content/drive/MyDrive/Capstone Project Huze/Dataset_fix.zip'

# Specify the directory where you want to extract the contents of the ZIP file
extract_dir = '/content'

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Optional: List the extracted files
extracted_files = zip_ref.namelist()
print("Extracted files:")
for file in extracted_files:
    print(file)

In [4]:
# gabung val_train_test
def move(path):
  dst_path = "/content/gabungnan"
  for class_folder in sorted(os.listdir(path)):
    class_path = os.path.join(path, class_folder)
    dst_path_folder = os.path.join(dst_path, class_folder)
    os.makedirs(dst_path_folder, exist_ok=True)
    image_files = os.listdir(class_path)
    for i, image_file in enumerate(image_files):
        src_path = os.path.join(class_path, image_file)
        file_path_gabungan = os.path.join(dst_path_folder, image_file)
        shutil.copy(src_path, file_path_gabungan)
move('/content/Dataset_fix/train_dir')
move('/content/Dataset_fix/validation_dir')
move('/content/Dataset_fix/test_dir')

In [5]:
# class in list
def get_list_class(path):
  arr = []
  for class_folder in sorted(os.listdir(path)):
    arr.append(class_folder)
  return arr
list_class_name = get_list_class("/content/gabungnan")

In [64]:
# def random_1(path):
#   for class_folder in sorted(os.listdir(path)):
#     class_path = os.path.join(path, class_folder)
#     image_files = os.listdir(class_path)
#     random.shuffle(image_files)
#     for i, image_file in enumerate(image_files[500:]):
#       image_path = os.path.join(class_path, image_file)
#       os.remove(image_path)
# random_1('/content/gabungnan')

In [6]:
model = load_model('/content/drive/MyDrive/Capstone Project Huze/Model Percobaan/Xception-Dataset_fix/Xception_datafix.h5')

In [7]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.xception import preprocess_input
from sklearn.metrics.pairwise import cosine_similarity

target_size = (299, 299)

dataset_dir = '/content/gabungnan'

def extract_features(image_path):
    img = image.load_img(image_path, target_size=target_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)
    features = np.squeeze(features)
    return features

# Load and preprocess the dataset
image_paths = []
category_labels = []

for category in os.listdir(dataset_dir):
    category_dir = os.path.join(dataset_dir, category)
    if os.path.isdir(category_dir):
        for image_file in os.listdir(category_dir):
            image_path = os.path.join(category_dir, image_file)
            features = extract_features(image_path)
            image_paths.append(image_path)
            category_labels.append(category)

# Convert the features and labels to numpy arrays
features = np.array([extract_features(image_path) for image_path in image_paths])



Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 37ms/step


In [97]:
category_labels = np.array(category_labels)
# Select a random image as the user's preferred image
liked_image_index = np.random.randint(len(image_paths))
liked_image_path = image_paths[liked_image_index]
liked_image_category = category_labels[liked_image_index]

# Reshape the features array to have 2 dimensions
reshaped_features = np.reshape(features, (features.shape[0], -1))

# Calculate the cosine similarity between the preferred image and other images
similarities = cosine_similarity([reshaped_features[liked_image_index]], reshaped_features).flatten()

# Sort the images based on similarity (excluding images from the same category)
sorted_indices = np.argsort(similarities)[::-1]
recommended_categories = []

for index in sorted_indices:
    if category_labels[index] != liked_image_category:
        recommended_categories.append(category_labels[index])
        if len(recommended_categories) == 5:
            break

# Print the result
print(f"Preferred Image: {liked_image_path}")
print("Recommended Categories:")
for i, category in enumerate(recommended_categories):
    print(f"{i+1}. {category}")

Preferred Image: /content/gabungnan/Newfoundland/n02111277_5955.jpg
Recommended Categories:
1. Scottish_fold
2. Pug
3. Scottish_fold
4. Pomeranian
5. British_Shorthair


In [98]:
tf.saved_model.save(model, 'recommend_system_100eachImages')


In [99]:
import numpy as np

# Save the features array
np.save('/content/features.npy', features)
